In [105]:
import json
import pickle
import pandas as pd
import datetime
import numpy as np
from itertools import groupby
from operator import attrgetter
import re
import pandas as pd
import matplotlib.pyplot as plt

base_dir = '../datasets/twitter/'


# Read Raw Twitter Data

In [106]:
tweets_following = pickle.loads(open(base_dir + 'states_tweets', 'rb').read())
date_start = pickle.loads(open(base_dir + 'actions_tweets', 'rb').read())

In [107]:
start = date_start['Bob'][2][5][0]

actions = {}
states = {}
real_actions = {}
for us in tweets_following:
    actions[us] = []
    states[us] = []
    real_actions[us] = []
    for i, t_f in enumerate(tweets_following[us]):
        found = False
        if t_f[0] < start:
            continue
        for j, t_n in enumerate(date_start[us][2]):
            if t_n[7] == t_f[1]:
                actions[us].append(t_f+[1])
                real_actions[us].append([1])
                states[us].append(t_f)
                found = True
            if found:
                break
        if not found:
            if np.random.random() < 0.01:
                actions[us].append(t_f+[0])
                real_actions[us].append([0])
                states[us].append(t_f)

In [108]:
statistics = {}
for us in actions:
    num_si = 0
    num_no = 0
    num_like_si = 0
    num_like_no = 0
    num_ret_si = 0
    num_ret_no = 0
    med_ret = 0
    med_like = 0
    num_tweet = 0
    for ac in actions[us]:
        num_tweet +=1
        med_like += ac[3]
        med_ret += ac[2]
        if ac[-1] == 0:
            num_no += 1
            num_like_no += ac[3]
            num_ret_no += ac[2]
        else:
            num_si += 1
            num_like_si += ac[3]
            num_ret_si += ac[2]
    statistics[us] = {'yes':num_si, 'no':num_no, 'num_ret_yes':num_ret_si, 'num_ret_no':num_ret_no, 
                      'num_like_yes':num_like_si, 'num_like_no':num_like_no}

In [109]:
rewards = {}
real_states = {}
max_pop = {}
for us in actions:
    if len(actions[us]) == 0:
        continue
    pop = actions[us][0][2] + actions[us][0][3]
    tot_delay = 0
    tot_pop = 0
    max_pop[us] = []
    #rewards[us] = [[0, pop]]
    rewards[us] = []
    real_states[us] = []
    last = actions[us][0][0]
    num_t_day = 0
    n_actions = 0
    last_tweet = -1
    for ac in actions[us]:
        n_actions += 1
        pop = ac[2] + ac[3]
        max_pop[us].append(pop)
        if n_actions > 10:
            n_actions = 1
            num_t_day = 0
            last_tweet = -1
        if last_tweet != -1:
            delta_t = -np.max((0,1-0.1*(n_actions-last_tweet)))
        else:
            delta_t = 0
        if ac[-1] == 1:
            num_t_day += 1
            last_tweet = n_actions            
            rewards[us].append([delta_t, pop, num_t_day/10 ])
        else:
            rewards[us].append([delta_t,0, num_t_day/10])
        real_states[us].append([delta_t, pop, num_t_day/10])


        tot_delay += (ac[0]-last).total_seconds()/60/60/24
        tot_delay += pop
        if ac[-1] == 1:
            last = ac[0]
    statistics[us]['pop'] = tot_pop/len(rewards[us])
    statistics[us]['delay'] = tot_delay/len(rewards[us])

In [110]:
for us in rewards:
    for a in range(len(rewards[us])):
        r = rewards[us][a]
        m = np.mean(max_pop[us])
        r[1] /= m
        r[1] = np.clip(r[1], 0, 1)
        rewards[us][a] = r
        s = real_states[us][a]
        s[1] /= m
        s[1] = np.clip(s[1], 0, 1)
        real_states[us][a] = s

In [111]:
real_actions = {}
for us in actions:
    real_actions[us] = []
    for act in actions[us]:
        real_actions[us].append(act[-1])

# Save the trajectories

In [113]:
pickle.dump(real_actions, open(base_dir + 'actions.pkl', 'wb'))
pickle.dump(rewards,open(base_dir + 'rewards.pkl', 'wb'))
pickle.dump(real_states,open(base_dir + 'states.pkl', 'wb'))